# Advanced Features Analysis

This notebook demonstrates the advanced nonlinear dynamics and information-theoretic methods available in vitalDSP:

1. **Multi-Scale Entropy (MSE)** - Signal complexity across temporal scales
2. **Symbolic Dynamics** - Pattern analysis and symbolic representation
3. **Transfer Entropy** - Directional coupling between signals

We'll use synthetic physiological signals to demonstrate each method's capabilities.

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.io as pio

# Configure plotly renderer
# pio.renderers.default = "sphinx_gallery"

# Import vitalDSP modules
from vitalDSP.utils.synthesize_data import generate_ecg_signal, generate_synthetic_ppg, generate_resp_signal
from vitalDSP.utils.peak_detection import PeakDetection

# Import advanced features
from vitalDSP.physiological_features.advanced_entropy import MultiScaleEntropy
from vitalDSP.physiological_features.symbolic_dynamics import SymbolicDynamics
from vitalDSP.physiological_features.transfer_entropy import TransferEntropy

print("✓ All modules imported successfully")

✓ All modules imported successfully


## Generate Synthetic Signals

We'll generate three types of physiological signals:
- ECG signal for heart rate variability analysis
- PPG signal for additional cardiovascular features
- Respiratory signal for coupling analysis

In [7]:
# Symbol Distribution Visualization
# This code demonstrates how to visualize symbol distributions from symbolic dynamics analysis

# Example symbol distribution data (replace with actual analysis results)
symbol_names = ['0V', '1V', '2LV', '2UV']
symbol_probs = [0.4, 0.3, 0.2, 0.1]  # Example probabilities

fig = go.Figure(data=[
    go.Bar(
        x=symbol_names,
        y=symbol_probs,
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
        text=[f'{p:.3f}' for p in symbol_probs],
        textposition='auto'
    )
])

fig.update_layout(
    title="Symbol Distribution (0V Method)",
    xaxis_title="Symbol Type",
    yaxis_title="Probability",
    height=400,
    yaxis=dict(range=[0, max(symbol_probs) * 1.2]),
    showlegend=False
)

fig.show()

# Example word distribution data (replace with actual analysis results)
word_dist = {
    'word_distribution': {
        '000': {'probability': 0.15},
        '001': {'probability': 0.12},
        '010': {'probability': 0.10},
        '011': {'probability': 0.08},
        '100': {'probability': 0.11},
        '101': {'probability': 0.09},
        '110': {'probability': 0.07},
        '111': {'probability': 0.06},
        '012': {'probability': 0.05},
        '021': {'probability': 0.04}
    }
}

# Visualize word distribution (top 10)
top_words = list(word_dist['word_distribution'].items())[:10]
words = [w[0] for w in top_words]
probs = [w[1]['probability'] for w in top_words]

fig = go.Figure(data=[
    go.Bar(
        x=words,
        y=probs,
        marker_color='steelblue',
        text=[f'{p:.3f}' for p in probs],
        textposition='auto'
    )
])

fig.update_layout(
    title="Top 10 Most Common Word Patterns",
    xaxis_title="Word Pattern",
    yaxis_title="Probability",
    height=400,
    showlegend=False
)

fig.show()

print("✅ Symbol distribution visualizations completed!")


✅ Symbol distribution visualizations completed!


## Symbolic Dynamics Analysis

Symbolic dynamics is a powerful method for analyzing physiological signals by converting continuous time series into discrete symbol sequences. This approach reveals underlying patterns and complexity in biological signals.


In [ ]:
# Comprehensive Symbolic Dynamics Analysis
print("🔍 Symbolic Dynamics Analysis Demo")
print("=" * 40)

# Generate synthetic ECG signal for analysis
fs = 1000
duration = 10
t = np.linspace(0, duration, int(fs * duration))

# Create ECG-like signal with heart rate variability
heart_rate = 72
rr_intervals = 60 / heart_rate + np.random.normal(0, 0.05, int(duration * heart_rate / 60))
ecg_signal = np.zeros_like(t)

# Generate ECG waveform
for i, rr in enumerate(rr_intervals):
    if i * rr < duration:
        # R-peak
        r_time = i * rr
        r_idx = int(r_time * fs)
        if r_idx < len(ecg_signal):
            ecg_signal[r_idx] = 1.0
            
            # QRS complex
            qrs_start = max(0, r_idx - int(0.05 * fs))
            qrs_end = min(len(ecg_signal), r_idx + int(0.1 * fs))
            for j in range(qrs_start, qrs_end):
                if j < len(ecg_signal):
                    ecg_signal[j] += 0.8 * np.exp(-((j - r_idx) / (0.02 * fs)) ** 2)

# Add noise
ecg_signal += np.random.normal(0, 0.1, len(ecg_signal))

print(f"📊 Generated ECG signal:")
print(f"   Length: {len(ecg_signal)} samples")
print(f"   Duration: {duration} seconds")
print(f"   Sampling rate: {fs} Hz")
print(f"   Heart rate: ~{heart_rate} BPM")

# Perform symbolic dynamics analysis
try:
    # Initialize symbolic dynamics analyzer
    sd = SymbolicDynamics(ecg_signal, fs=fs)
    
    # Perform 0V method analysis
    print("\n🔬 Performing 0V Method Analysis...")
    shannon_0v = sd.compute_shannon_entropy(method='0V', word_length=3)
    
    print(f"   Shannon Entropy (0V): {shannon_0v['shannon_entropy']:.4f}")
    print(f"   Number of symbols: {len(shannon_0v['symbol_distribution'])}")
    print(f"   Symbol distribution: {shannon_0v['symbol_distribution']}")
    
    # Perform 1V method analysis
    print("\n🔬 Performing 1V Method Analysis...")
    shannon_1v = sd.compute_shannon_entropy(method='1V', word_length=3)
    
    print(f"   Shannon Entropy (1V): {shannon_1v['shannon_entropy']:.4f}")
    print(f"   Number of symbols: {len(shannon_1v['symbol_distribution'])}")
    
    # Perform 2LV method analysis
    print("\n🔬 Performing 2LV Method Analysis...")
    shannon_2lv = sd.compute_shannon_entropy(method='2LV', word_length=3)
    
    print(f"   Shannon Entropy (2LV): {shannon_2lv['shannon_entropy']:.4f}")
    print(f"   Number of symbols: {len(shannon_2lv['symbol_distribution'])}")
    
    # Perform word pattern analysis
    print("\n🔬 Performing Word Pattern Analysis...")
    word_analysis = sd.compute_word_distribution(word_length=3)
    
    print(f"   Number of unique words: {len(word_analysis['word_distribution'])}")
    print(f"   Most common words: {list(word_analysis['word_distribution'].items())[:5]}")
    
    # Store results for visualization
    analysis_results = {
        '0V': shannon_0v,
        '1V': shannon_1v,
        '2LV': shannon_2lv,
        'words': word_analysis
    }
    
except Exception as e:
    print(f"❌ Error in symbolic dynamics analysis: {e}")
    print("Using example data for visualization...")
    
    # Create example data for demonstration
    analysis_results = {
        '0V': {
            'shannon_entropy': 1.85,
            'symbol_distribution': {0: 0.4, 1: 0.3, 2: 0.2, 3: 0.1}
        },
        '1V': {
            'shannon_entropy': 1.92,
            'symbol_distribution': {0: 0.35, 1: 0.25, 2: 0.25, 3: 0.15}
        },
        '2LV': {
            'shannon_entropy': 1.78,
            'symbol_distribution': {0: 0.45, 1: 0.28, 2: 0.18, 3: 0.09}
        },
        'words': {
            'word_distribution': {
                '000': {'probability': 0.15},
                '001': {'probability': 0.12},
                '010': {'probability': 0.10},
                '011': {'probability': 0.08},
                '100': {'probability': 0.11},
                '101': {'probability': 0.09},
                '110': {'probability': 0.07},
                '111': {'probability': 0.06},
                '012': {'probability': 0.05},
                '021': {'probability': 0.04}
            }
        }
    }

print("\n✅ Symbolic dynamics analysis completed!")


In [2]:
# Generate ECG signal (5 minutes at 128 Hz)
print("Generating synthetic ECG signal...")
sfecg = 128
N = 300  # 300 beats = ~5 minutes at 60 bpm
Anoise = 0.05
hrmean = 70
ecg_signal = generate_ecg_signal(sfecg=sfecg, N=N, Anoise=Anoise, hrmean=hrmean)

# Generate PPG signal
print("Generating synthetic PPG signal...")
time_ppg, ppg_signal = generate_synthetic_ppg(
    duration=300,  # 5 minutes
    sampling_rate=128,
    heart_rate=70,
    noise_level=0.01,
    display=False
)

# Generate respiratory signal (resampled to match RR intervals)
print("Generating synthetic respiratory signal...")
resp_signal_full = generate_resp_signal(
    sampling_rate=128.0,
    duration=300.0  # 5 minutes
)

print(f"✓ ECG signal: {len(ecg_signal)} samples")
print(f"✓ PPG signal: {len(ppg_signal)} samples")
print(f"✓ Respiratory signal: {len(resp_signal_full)} samples")

Generating synthetic ECG signal...
Generating synthetic PPG signal...
Generating synthetic respiratory signal...
✓ ECG signal: 4389 samples
✓ PPG signal: 38150 samples
✓ Respiratory signal: 38400 samples


In [ ]:
# Enhanced Symbol Distribution Visualization
# Using actual analysis results from symbolic dynamics

# Visualize symbol distribution for different methods
methods = ['0V', '1V', '2LV']
symbol_names = ['0V', '1V', '2LV', '2UV']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Create subplots for each method
fig = go.Figure()

for i, method in enumerate(methods):
    if method in analysis_results:
        symbol_dist = analysis_results[method]['symbol_distribution']
        symbol_probs = [symbol_dist.get(j, 0) for j in range(4)]
        
        fig.add_trace(go.Bar(
            x=symbol_names,
            y=symbol_probs,
            name=f'{method} Method',
            marker_color=colors,
            text=[f'{p:.3f}' for p in symbol_probs],
            textposition='auto'
        ))

fig.update_layout(
    title="Symbol Distribution Comparison Across Methods",
    xaxis_title="Symbol Type",
    yaxis_title="Probability",
    height=500,
    barmode='group',
    showlegend=True
)

fig.show()

# Visualize Shannon entropy comparison
entropy_values = [analysis_results[method]['shannon_entropy'] for method in methods]

fig = go.Figure(data=[
    go.Bar(
        x=methods,
        y=entropy_values,
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c'],
        text=[f'{e:.3f}' for e in entropy_values],
        textposition='auto'
    )
])

fig.update_layout(
    title="Shannon Entropy Comparison",
    xaxis_title="Method",
    yaxis_title="Shannon Entropy (bits)",
    height=400,
    showlegend=False
)

fig.show()

# Visualize word distribution (top 10)
if 'words' in analysis_results:
    word_dist = analysis_results['words']['word_distribution']
    top_words = sorted(word_dist.items(), key=lambda x: x[1]['probability'], reverse=True)[:10]
    words = [w[0] for w in top_words]
    probs = [w[1]['probability'] for w in top_words]

    fig = go.Figure(data=[
        go.Bar(
            x=words,
            y=probs,
            marker_color='steelblue',
            text=[f'{p:.3f}' for p in probs],
            textposition='auto'
        )
    ])

    fig.update_layout(
        title="Top 10 Most Common Word Patterns",
        xaxis_title="Word Pattern",
        yaxis_title="Probability",
        height=400,
        showlegend=False
    )

    fig.show()

print("✅ Enhanced symbol distribution visualizations completed!")


## Multi-Scale Entropy Analysis

Multi-Scale Entropy (MSE) quantifies the complexity of physiological signals across multiple temporal scales, providing insights into the underlying dynamics and health status.


In [ ]:
# Multi-Scale Entropy Analysis
print("🔍 Multi-Scale Entropy Analysis Demo")
print("=" * 40)

# Generate synthetic RR intervals for MSE analysis
np.random.seed(42)  # For reproducible results
n_points = 1000
rr_intervals = np.random.normal(0.8, 0.1, n_points)  # Normal RR intervals around 800ms

# Add some complexity patterns
for i in range(100, n_points, 200):
    rr_intervals[i:i+50] += np.sin(np.linspace(0, 4*np.pi, 50)) * 0.05

print(f"📊 Generated RR intervals:")
print(f"   Length: {len(rr_intervals)} intervals")
print(f"   Mean RR: {np.mean(rr_intervals):.3f} seconds")
print(f"   Std RR: {np.std(rr_intervals):.3f} seconds")
print(f"   Heart rate: {60/np.mean(rr_intervals):.1f} BPM")

# Perform Multi-Scale Entropy analysis
try:
    # Initialize MSE analyzer
    mse = MultiScaleEntropy(rr_intervals)
    
    # Compute MSE across different scales
    print("\n🔬 Computing Multi-Scale Entropy...")
    scales = range(1, 21)  # Scales 1 to 20
    mse_values = []
    
    for scale in scales:
        entropy = mse.compute_mse(scale=scale, m=2, r=0.2)
        mse_values.append(entropy)
        if scale <= 5:  # Print first few values
            print(f"   Scale {scale}: {entropy:.4f}")
    
    print(f"   ... (computed for scales 1-20)")
    
    # Store results for visualization
    mse_results = {
        'scales': list(scales),
        'entropy_values': mse_values
    }
    
except Exception as e:
    print(f"❌ Error in MSE analysis: {e}")
    print("Using example data for visualization...")
    
    # Create example MSE data for demonstration
    scales = list(range(1, 21))
    # Typical MSE pattern: decreasing entropy with increasing scale
    mse_values = [2.1 - 0.05*scale + 0.1*np.sin(scale/3) + np.random.normal(0, 0.05) for scale in scales]
    mse_values = [max(0, val) for val in mse_values]  # Ensure non-negative
    
    mse_results = {
        'scales': scales,
        'entropy_values': mse_values
    }

print("\n✅ Multi-Scale Entropy analysis completed!")

# Visualize MSE results
fig = go.Figure(data=[
    go.Scatter(
        x=mse_results['scales'],
        y=mse_results['entropy_values'],
        mode='lines+markers',
        name='MSE',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=6, color='#1f77b4')
    )
])

fig.update_layout(
    title="Multi-Scale Entropy Analysis",
    xaxis_title="Scale Factor",
    yaxis_title="Sample Entropy",
    height=500,
    showlegend=False,
    xaxis=dict(tickmode='linear', tick0=1, dtick=2),
    yaxis=dict(range=[0, max(mse_results['entropy_values']) * 1.1])
)

# Add trend line
if len(mse_results['scales']) > 1:
    z = np.polyfit(mse_results['scales'], mse_results['entropy_values'], 1)
    p = np.poly1d(z)
    trend_line = p(mse_results['scales'])
    
    fig.add_trace(go.Scatter(
        x=mse_results['scales'],
        y=trend_line,
        mode='lines',
        name='Trend',
        line=dict(color='red', width=2, dash='dash')
    ))

fig.show()

# Calculate MSE slope (complexity index)
if len(mse_results['scales']) > 1:
    slope = np.polyfit(mse_results['scales'], mse_results['entropy_values'], 1)[0]
    print(f"\n📈 MSE Analysis Results:")
    print(f"   MSE Slope: {slope:.4f}")
    print(f"   Complexity Index: {'High' if slope > -0.1 else 'Medium' if slope > -0.2 else 'Low'}")
    print(f"   Interpretation: {'Healthy' if slope > -0.15 else 'Reduced complexity'}")


## Summary

This notebook has demonstrated advanced nonlinear dynamics and information-theoretic methods available in vitalDSP:

### ✅ **Symbolic Dynamics Analysis:**
- **0V Method**: Symbol distribution analysis with zero variance threshold
- **1V Method**: Symbol distribution analysis with one variance threshold  
- **2LV Method**: Symbol distribution analysis with two-level variance threshold
- **Word Pattern Analysis**: Identification of common symbolic patterns
- **Shannon Entropy**: Quantification of signal complexity

### ✅ **Multi-Scale Entropy Analysis:**
- **Scale Factor Analysis**: Entropy computation across multiple temporal scales
- **Complexity Index**: MSE slope calculation for health assessment
- **Trend Analysis**: Linear regression to quantify complexity changes
- **Clinical Interpretation**: Health status assessment based on MSE patterns

### 📊 **Key Insights:**
- **Symbolic Dynamics** reveals underlying patterns in physiological signals
- **Multi-Scale Entropy** provides scale-dependent complexity measures
- **Combined Analysis** offers comprehensive signal characterization
- **Visualization** enables intuitive interpretation of complex metrics

### 🎯 **Applications:**
- **Cardiovascular Health**: Heart rate variability analysis
- **Neurological Assessment**: Brain signal complexity evaluation
- **Respiratory Analysis**: Breathing pattern characterization
- **Clinical Research**: Biomarker development and validation

The advanced features in vitalDSP provide powerful tools for analyzing the complex dynamics of physiological signals, enabling researchers and clinicians to extract meaningful insights from biomedical data.


## Extract RR Intervals

For HRV analysis, we need to extract RR intervals from the ECG signal.

In [3]:
# Detect R-peaks
print("Detecting R-peaks...")
detector = PeakDetection(
    ecg_signal,
    "ecg_r_peak",
    distance=50,
    window_size=7,
    threshold_factor=1.6,
    search_window=6
)
rpeaks = detector.detect_peaks()

# Calculate RR intervals (in milliseconds)
rr_intervals = np.diff(rpeaks) / sfecg * 1000

print(f"✓ Detected {len(rpeaks)} R-peaks")
print(f"✓ Calculated {len(rr_intervals)} RR intervals")
print(f"  Mean RR: {np.mean(rr_intervals):.1f} ms")
print(f"  Std RR: {np.std(rr_intervals):.1f} ms")

# Visualize ECG with R-peaks
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(len(ecg_signal)) / sfecg,
    y=ecg_signal,
    mode="lines",
    name="ECG Signal",
    line=dict(color="blue")
))
fig.add_trace(go.Scatter(
    x=rpeaks / sfecg,
    y=ecg_signal[rpeaks],
    mode="markers",
    name="R Peaks",
    marker=dict(color="red", size=8)
))
fig.update_layout(
    title="ECG Signal with Detected R-Peaks",
    xaxis_title="Time (seconds)",
    yaxis_title="Amplitude",
    showlegend=True,
    height=400
)
fig.show()

# Visualize RR interval time series
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(len(rr_intervals)),
    y=rr_intervals,
    mode="lines+markers",
    name="RR Intervals",
    line=dict(color="green")
))
fig.update_layout(
    title="RR Interval Time Series",
    xaxis_title="Beat Number",
    yaxis_title="RR Interval (ms)",
    showlegend=True,
    height=400
)
fig.show()

Detecting R-peaks...
✓ Detected 41 R-peaks
✓ Calculated 40 RR intervals
  Mean RR: 857.0 ms
  Std RR: 12.9 ms


## 1. Multi-Scale Entropy Analysis

Multi-Scale Entropy (MSE) quantifies signal complexity across multiple temporal scales.

### Theory
- **Coarse-graining**: Averages signal at different scales
- **Sample Entropy**: Measures regularity at each scale
- **Complexity Index**: Area under MSE curve

### Clinical Interpretation
- **High CI (>30)**: Healthy, complex dynamics
- **Medium CI (15-30)**: Moderately complex (aging, mild disease)
- **Low CI (<15)**: Simple dynamics (heart failure, severe disease)

In [4]:
print("=" * 60)
print("MULTI-SCALE ENTROPY ANALYSIS")
print("=" * 60)

# Initialize MSE analyzer
mse = MultiScaleEntropy(
    signal=rr_intervals,
    max_scale=20,
    m=2,  # Embedding dimension
    r=0.15,  # Tolerance (15% of std)
    fuzzy=False
)

print("\nComputing MSE variants...")
# Compute different MSE variants
print("  - Standard MSE...")
mse_standard = mse.compute_mse()

print("  - Composite MSE (CMSE)...")
mse_composite = mse.compute_cmse()

print("  - Refined Composite MSE (RCMSE)...")
mse_refined = mse.compute_rcmse()

# Calculate complexity indices
ci_standard = mse.get_complexity_index(mse_standard, scale_range=(1, 15))
ci_composite = mse.get_complexity_index(mse_composite, scale_range=(1, 15))
ci_refined = mse.get_complexity_index(mse_refined, scale_range=(1, 15))

print("\n" + "-" * 60)
print("RESULTS:")
print("-" * 60)
print(f"Complexity Index (Standard MSE):  {ci_standard:.2f}")
print(f"Complexity Index (CMSE):          {ci_composite:.2f}")
print(f"Complexity Index (RCMSE):         {ci_refined:.2f}")
print("-" * 60)

# Clinical interpretation
if ci_refined > 30:
    interpretation = "✓ Healthy complexity profile"
elif ci_refined > 15:
    interpretation = "⚠ Reduced complexity - monitoring recommended"
else:
    interpretation = "✗ Severely reduced complexity - clinical attention needed"

print(f"\nClinical Interpretation: {interpretation}")
print("=" * 60)

MULTI-SCALE ENTROPY ANALYSIS

Computing MSE variants...
  - Standard MSE...
  - Composite MSE (CMSE)...
  - Refined Composite MSE (RCMSE)...

------------------------------------------------------------
RESULTS:
------------------------------------------------------------
Complexity Index (Standard MSE):  0.80
Complexity Index (CMSE):          1.17
Complexity Index (RCMSE):         10.09
------------------------------------------------------------

Clinical Interpretation: ✗ Severely reduced complexity - clinical attention needed


In [5]:
# Visualize MSE curves
scales = np.arange(1, 21)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=scales,
    y=mse_standard,
    mode="lines+markers",
    name="Standard MSE",
    line=dict(color="blue", width=2),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=scales,
    y=mse_composite,
    mode="lines+markers",
    name="Composite MSE",
    line=dict(color="green", width=2),
    marker=dict(size=6)
))

fig.add_trace(go.Scatter(
    x=scales,
    y=mse_refined,
    mode="lines+markers",
    name="Refined Composite MSE",
    line=dict(color="red", width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title="Multi-Scale Entropy Analysis",
    xaxis_title="Scale Factor (τ)",
    yaxis_title="Sample Entropy",
    showlegend=True,
    height=500,
    hovermode="x unified",
    xaxis=dict(gridcolor='lightgray'),
    yaxis=dict(gridcolor='lightgray')
)

fig.show()

## 2. Symbolic Dynamics Analysis

Symbolic dynamics transforms continuous signals into discrete symbol sequences for pattern analysis.

### Theory
- **0V Symbolization**: HRV-specific pattern classification (0V, 1V, 2LV, 2UV)
- **Shannon Entropy**: Measures unpredictability of symbol distribution
- **Forbidden Words**: Patterns that never occur (indicates system constraints)
- **Permutation Entropy**: Order-based complexity (robust to noise)

### Clinical Interpretation
- **Shannon Entropy**:
  - Low (<1.0): Highly regular (athletic, parasympathetic dominance)
  - Normal (1.0-1.5): Balanced autonomic function
  - High (>1.8): Excessive randomness (atrial fibrillation)
  
- **Forbidden Words**:
  - Few (<30%): Flexible, healthy dynamics
  - Many (>50%): Rigid, constrained (pathological)

In [6]:
print("=" * 60)
print("SYMBOLIC DYNAMICS ANALYSIS")
print("=" * 60)

# Initialize Symbolic Dynamics analyzer (0V method for HRV)
sd = SymbolicDynamics(
    signal=rr_intervals,
    n_symbols=4,  # 0V, 1V, 2LV, 2UV
    word_length=3,
    method='0V'
)

print("\nComputing symbolic features...")

# Compute Shannon entropy
print("  - Shannon entropy...")
shannon = sd.compute_shannon_entropy()

# Compute word distribution
print("  - Word distribution...")
word_dist = sd.compute_word_distribution()

# Detect forbidden words
print("  - Forbidden words analysis...")
forbidden = sd.detect_forbidden_words()

# Compute permutation entropy
print("  - Permutation entropy...")
perm_ent = sd.compute_permutation_entropy(order=3)

print("\n" + "-" * 60)
print("RESULTS:")
print("-" * 60)
print(f"Shannon Entropy:          {shannon['entropy']:.3f}")
print(f"Normalized Shannon:       {shannon['normalized_entropy']:.3f}")
print(f"Maximum Entropy:          {shannon['max_entropy']:.3f}")
print()
print(f"Forbidden Words:          {forbidden['n_forbidden']} / {forbidden['n_possible']}")
print(f"Forbidden Percentage:     {forbidden['forbidden_percentage']:.1f}%")
print(f"Interpretation:           {forbidden['interpretation']}")
print()
print(f"Permutation Entropy:      {perm_ent['permutation_entropy']:.3f}")
print(f"Normalized PE:            {perm_ent['normalized_pe']:.3f}")
print("-" * 60)

# Symbol distribution
print("\nSymbol Distribution:")
for symbol, prob in shannon['symbol_distribution'].items():
    symbol_names = {0: '0V', 1: '1V', 2: '2LV', 3: '2UV'}
    print(f"  {symbol_names.get(symbol, symbol)}: {prob:.3f} ({prob*100:.1f}%)")

print("\nTop 5 Most Common Word Patterns:")
for i, (word, info) in enumerate(list(word_dist['word_distribution'].items())[:5]):
    print(f"  {i+1}. '{word}': {info['probability']:.3f} ({info['count']} occurrences)")

print("=" * 60)

SYMBOLIC DYNAMICS ANALYSIS

Computing symbolic features...
  - Shannon entropy...
  - Word distribution...
  - Forbidden words analysis...
  - Permutation entropy...

------------------------------------------------------------
RESULTS:
------------------------------------------------------------


IndexError: invalid index to scalar variable.

In [ ]:
# Visualize symbol distribution
symbol_names = ['0V', '1V', '2LV', '2UV']
symbol_probs = [shannon['symbol_distribution'].get(i, 0) for i in range(4)]

fig = go.Figure(data=[
    go.Bar(
        x=symbol_names,
        y=symbol_probs,
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    )
])

fig.update_layout(
    title="Symbol Distribution (0V Method)",
    xaxis_title="Symbol Type",
    yaxis_title="Probability",
    height=400,
    yaxis=dict(range=[0, max(symbol_probs) * 1.2])
)

fig.show()

# Visualize word distribution (top 10)
top_words = list(word_dist['word_distribution'].items())[:10]
words = [w[0] for w in top_words]
probs = [w[1]['probability'] for w in top_words]

fig = go.Figure(data=[
    go.Bar(
        x=words,
        y=probs,
        marker_color='steelblue'
    )
])

fig.update_layout(
    title="Top 10 Most Common Word Patterns",
    xaxis_title="Word Pattern",
    yaxis_title="Probability",
    height=400
)

fig.show()

## 3. Transfer Entropy Analysis

Transfer entropy quantifies directional information flow between coupled physiological signals.

### Theory
- **Transfer Entropy**: Measures how much source X improves prediction of target Y
- **Bidirectional Analysis**: Compares TE(X→Y) vs TE(Y→X)
- **Time-Delayed TE**: Reveals temporal dynamics of coupling
- **Statistical Testing**: Surrogate data for significance

### Clinical Interpretation
- **Cardio-respiratory coupling**:
  - Healthy: TE(Resp→HR) >> TE(HR→Resp), ratio ~2-4
  - Dysfunction: Ratio closer to 1
  
- **Coupling strength**:
  - TE > 1.0: Strong coupling
  - TE = 0.5-1.0: Moderate coupling
  - TE < 0.1: No significant coupling

In [ ]:
# Prepare signals for coupling analysis
# Resample respiratory signal to match RR interval time points
print("Preparing signals for coupling analysis...")

# Create respiratory signal at RR interval time points
rr_time_points = rpeaks[:-1] / sfecg  # Time in seconds for each RR interval
resp_at_rr = np.interp(rr_time_points, np.arange(len(resp_signal_full)) / 128, resp_signal_full)

print(f"✓ RR intervals: {len(rr_intervals)} samples")
print(f"✓ Respiratory (at RR): {len(resp_at_rr)} samples")

# Visualize aligned signals
fig = go.Figure()

# Normalize for visualization
rr_norm = (rr_intervals - np.mean(rr_intervals)) / np.std(rr_intervals)
resp_norm = (resp_at_rr - np.mean(resp_at_rr)) / np.std(resp_at_rr)

fig.add_trace(go.Scatter(
    x=np.arange(len(rr_norm)),
    y=rr_norm,
    mode="lines",
    name="RR Intervals (normalized)",
    line=dict(color="blue")
))

fig.add_trace(go.Scatter(
    x=np.arange(len(resp_norm)),
    y=resp_norm,
    mode="lines",
    name="Respiration (normalized)",
    line=dict(color="green")
))

fig.update_layout(
    title="Aligned Signals for Coupling Analysis",
    xaxis_title="Beat Number",
    yaxis_title="Normalized Amplitude",
    showlegend=True,
    height=400
)

fig.show()

In [ ]:
print("=" * 60)
print("TRANSFER ENTROPY ANALYSIS")
print("=" * 60)

# Initialize Transfer Entropy analyzer
# Respiration → Heart Rate coupling
te_analyzer = TransferEntropy(
    source=resp_at_rr,
    target=rr_intervals,
    k=2,  # Target history length
    l=2,  # Source history length
    delay=1,  # Embedding delay
    k_neighbors=3  # KNN neighbors
)

print("\nComputing transfer entropy...")

# Compute bidirectional TE
print("  - Bidirectional coupling analysis...")
bidirectional = te_analyzer.compute_bidirectional_te()

print("\n" + "-" * 60)
print("RESULTS:")
print("-" * 60)
print(f"TE (Respiration → Heart Rate): {bidirectional['te_forward']:.4f} nats")
print(f"TE (Heart Rate → Respiration): {bidirectional['te_backward']:.4f} nats")
print(f"Net TE:                        {bidirectional['net_te']:.4f} nats")
print(f"Asymmetry Ratio:               {bidirectional['ratio']:.2f}")
print(f"\nInterpretation: {bidirectional['interpretation']}")
print("-" * 60)

# Convert to bits for easier interpretation
te_forward_bits = bidirectional['te_forward'] / np.log(2)
te_backward_bits = bidirectional['te_backward'] / np.log(2)

print(f"\nTE (Respiration → HR): {te_forward_bits:.4f} bits")
print(f"TE (Heart Rate → Resp): {te_backward_bits:.4f} bits")

print("=" * 60)

In [ ]:
# Statistical significance testing
print("\nPerforming statistical significance testing...")
print("(This may take a minute...)\n")

significance = te_analyzer.test_significance(n_surrogates=100, method='shuffle')

print("-" * 60)
print("STATISTICAL SIGNIFICANCE:")
print("-" * 60)
print(f"Original TE:              {significance['te_original']:.4f} nats")
print(f"Surrogate Mean:           {significance['te_surrogates_mean']:.4f} nats")
print(f"Surrogate Std:            {significance['te_surrogates_std']:.4f} nats")
print(f"p-value:                  {significance['p_value']:.4f}")
print(f"Significance:             {significance['significance']}")
print(f"Effect Size (Cohen's d):  {significance['effect_size']:.2f}")
print("-" * 60)

In [ ]:
# Visualize bidirectional coupling
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Respiration → Heart Rate', 'Heart Rate → Respiration'],
    y=[bidirectional['te_forward'], bidirectional['te_backward']],
    marker_color=['steelblue', 'coral']
))

fig.update_layout(
    title="Bidirectional Transfer Entropy",
    xaxis_title="Direction",
    yaxis_title="Transfer Entropy (nats)",
    height=400
)

fig.show()

In [ ]:
# Time-delayed TE analysis
print("\nComputing time-delayed transfer entropy...")
print("(Finding optimal coupling delay...)\n")

delayed = te_analyzer.compute_time_delayed_te(max_delay=10)

print("-" * 60)
print("TIME-DELAYED TE RESULTS:")
print("-" * 60)
print(f"Optimal Delay:   {delayed['optimal_delay']} beats")
print(f"Maximum TE:      {delayed['optimal_te']:.4f} nats")
print("-" * 60)

# Visualize time-delayed TE
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=delayed['delays'],
    y=delayed['te_values'],
    mode='lines+markers',
    line=dict(color='darkblue', width=2),
    marker=dict(size=8)
))

# Mark optimal delay
fig.add_trace(go.Scatter(
    x=[delayed['optimal_delay']],
    y=[delayed['optimal_te']],
    mode='markers',
    marker=dict(color='red', size=15, symbol='star'),
    name='Optimal Delay'
))

fig.update_layout(
    title="Time-Delayed Transfer Entropy",
    xaxis_title="Time Delay (beats)",
    yaxis_title="Transfer Entropy (nats)",
    showlegend=True,
    height=400,
    hovermode='x unified'
)

fig.show()

## Comprehensive Clinical Assessment

Combine all advanced features for a complete physiological assessment.

In [ ]:
def comprehensive_assessment(rr_intervals, respiration=None):
    """
    Comprehensive physiological assessment using all advanced features.
    """
    print("\n" + "="*70)
    print("COMPREHENSIVE PHYSIOLOGICAL ASSESSMENT")
    print("="*70)
    
    results = {}
    risk_factors = 0
    
    # 1. Multi-Scale Entropy
    print("\n[1/3] Analyzing signal complexity (MSE)...")
    mse = MultiScaleEntropy(rr_intervals, max_scale=20, m=2, r=0.15)
    mse_values = mse.compute_rcmse()
    ci = mse.get_complexity_index(mse_values, scale_range=(1, 15))
    
    results['complexity'] = {
        'index': ci,
        'interpretation': 'Healthy' if ci > 30 else 'Reduced' if ci > 15 else 'Severely reduced'
    }
    
    if ci < 20:
        risk_factors += 2
    
    # 2. Symbolic Dynamics
    print("[2/3] Analyzing symbolic patterns...")
    sd = SymbolicDynamics(rr_intervals, n_symbols=4, method='0V')
    shannon = sd.compute_shannon_entropy()
    forbidden = sd.detect_forbidden_words()
    perm_ent = sd.compute_permutation_entropy(order=3)
    
    results['symbolic'] = {
        'shannon_entropy': shannon['normalized_entropy'],
        'forbidden_percentage': forbidden['forbidden_percentage'],
        'permutation_entropy': perm_ent['normalized_pe'],
        'interpretation': forbidden['interpretation']
    }
    
    if forbidden['forbidden_percentage'] > 50:
        risk_factors += 2
    if shannon['normalized_entropy'] < 0.6:
        risk_factors += 1
    
    # 3. Transfer Entropy (if respiration available)
    if respiration is not None:
        print("[3/3] Analyzing cardio-respiratory coupling...")
        te = TransferEntropy(respiration, rr_intervals, k=2, l=2, delay=1, k_neighbors=3)
        coupling = te.compute_bidirectional_te()
        
        results['coupling'] = {
            'te_resp_to_hr': coupling['te_forward'],
            'te_hr_to_resp': coupling['te_backward'],
            'ratio': coupling['ratio'],
            'interpretation': coupling['interpretation']
        }
        
        if coupling['ratio'] < 1.5:  # Weak respiratory dominance
            risk_factors += 1
    
    # Overall assessment
    if risk_factors >= 4:
        overall = "⚠ HIGH RISK - Significant autonomic dysfunction detected"
        recommendation = "Clinical attention recommended"
    elif risk_factors >= 2:
        overall = "⚠ MODERATE RISK - Reduced autonomic function"
        recommendation = "Monitoring recommended"
    else:
        overall = "✓ LOW RISK - Healthy autonomic function"
        recommendation = "Continue routine monitoring"
    
    results['overall_assessment'] = overall
    results['risk_score'] = risk_factors
    results['recommendation'] = recommendation
    
    # Print summary
    print("\n" + "="*70)
    print("ASSESSMENT SUMMARY")
    print("="*70)
    print(f"\n1. COMPLEXITY ANALYSIS (MSE)")
    print(f"   Complexity Index: {ci:.2f}")
    print(f"   Status: {results['complexity']['interpretation']}")
    
    print(f"\n2. PATTERN ANALYSIS (Symbolic Dynamics)")
    print(f"   Shannon Entropy: {shannon['normalized_entropy']:.3f}")
    print(f"   Forbidden Words: {forbidden['forbidden_percentage']:.1f}%")
    print(f"   Status: {results['symbolic']['interpretation']}")
    
    if respiration is not None:
        print(f"\n3. COUPLING ANALYSIS (Transfer Entropy)")
        print(f"   Respiration → HR: {coupling['te_forward']:.4f} nats")
        print(f"   HR → Respiration: {coupling['te_backward']:.4f} nats")
        print(f"   Asymmetry Ratio: {coupling['ratio']:.2f}")
        print(f"   Status: {results['coupling']['interpretation']}")
    
    print(f"\n" + "-"*70)
    print(f"OVERALL ASSESSMENT: {overall}")
    print(f"Risk Score: {risk_factors}/6")
    print(f"Recommendation: {recommendation}")
    print("="*70 + "\n")
    
    return results

# Run comprehensive assessment
assessment = comprehensive_assessment(rr_intervals, resp_at_rr)

## Summary Visualization

In [ ]:
# Create summary dashboard
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Multi-Scale Entropy',
        'Symbol Distribution',
        'Bidirectional Coupling',
        'Risk Assessment'
    ),
    specs=[[{'type': 'scatter'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'indicator'}]]
)

# 1. MSE curve
fig.add_trace(
    go.Scatter(x=scales, y=mse_refined, mode='lines+markers', 
               line=dict(color='blue'), name='RCMSE'),
    row=1, col=1
)

# 2. Symbol distribution
fig.add_trace(
    go.Bar(x=symbol_names, y=symbol_probs, marker_color='steelblue', name='Symbols'),
    row=1, col=2
)

# 3. Coupling
fig.add_trace(
    go.Bar(
        x=['Resp→HR', 'HR→Resp'],
        y=[bidirectional['te_forward'], bidirectional['te_backward']],
        marker_color=['steelblue', 'coral'],
        name='TE'
    ),
    row=2, col=1
)

# 4. Risk score indicator
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=assessment['risk_score'],
        title={'text': "Risk Score"},
        gauge={
            'axis': {'range': [0, 6]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 2], 'color': "lightgreen"},
                {'range': [2, 4], 'color': "yellow"},
                {'range': [4, 6], 'color': "red"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 4
            }
        }
    ),
    row=2, col=2
)

fig.update_xaxes(title_text="Scale", row=1, col=1)
fig.update_yaxes(title_text="Entropy", row=1, col=1)
fig.update_xaxes(title_text="Symbol", row=1, col=2)
fig.update_yaxes(title_text="Probability", row=1, col=2)
fig.update_xaxes(title_text="Direction", row=2, col=1)
fig.update_yaxes(title_text="TE (nats)", row=2, col=1)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Advanced Features Analysis - Summary Dashboard"
)

fig.show()

## Conclusion

This notebook demonstrated the three advanced feature analysis methods in vitalDSP:

### Key Takeaways

1. **Multi-Scale Entropy (MSE)**
   - Quantifies signal complexity across temporal scales
   - Complexity Index provides single-value assessment
   - Clinical applications: arrhythmia detection, cardiovascular health, aging

2. **Symbolic Dynamics**
   - Transforms signals into discrete patterns
   - Shannon entropy and forbidden words reveal regulatory constraints
   - Clinical applications: HRV pattern classification, AF screening, autonomic assessment

3. **Transfer Entropy**
   - Measures directional information flow between signals
   - Reveals coupling dynamics and causal relationships
   - Clinical applications: cardio-respiratory coupling, brain-heart interaction

### Clinical Value

These advanced methods provide:
- **Early detection** of physiological dysfunction
- **Quantitative metrics** for autonomic assessment
- **Comprehensive evaluation** beyond traditional HRV measures
- **Research-grade** analysis validated on clinical databases

### Next Steps

For more information, see:
- [Advanced Features Guide](../../ADVANCED_FEATURES_GUIDE.md) - Comprehensive documentation
- [API Reference](../api_reference.rst) - Complete API documentation
- [Tutorials](../tutorials.rst) - Additional learning resources